<a href="https://colab.research.google.com/github/adrianvallsc/FLAIR_stroke_segmentation/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# nnUNET para la segmentación de imágenes FLAIR de ictus

En este proyecto intentaremos entrenar una nnUNet para la segmentación automática de imágenes en FLAIR.

nnUNET es un paradigma de redes que permite el entrenamiento de redes profundas con arquitectura U-Net, la cual se ha demostrado ser de gran utilidad a la hora de segmentar imágenes médicas.

Para el entrenamiento de estas redes haremos uso GPUs como recursos computacionales. El actual cuaderno está pensado para ser ejecutado en Google Colab, aunque puede adapatarse para otros recursos


In [1]:
# Montamos google Colab, donde ya tenemos los datos organizados

from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## Instalación de nnUNet

Para la instalación de nnUNet, en primer lugar debemos instalar el paquete. En mi caso, el paquete lo tengo descargado dentro de mi Drive, por lo que no necesito descargarlo

In [ ]:
import os

# Necesitamos definir una serie de paths

# Base path = donde se almacenan las imágenes
# nnUNetFrame = donde se guardarán todos los modelos
# Debe de tener la siguiente estructura
# nnUNetFrame
# ├── dataset
#   ├── nnUNet_raw
#   ├── nnUNet_preprocessed
#   ├── nnUNet_trained_models

# Modificar según interese en el
wd = "/content/drive/MyDrive"
foldername = "Dataset001_Stroke"

nnFrame = os.path.join(wd, "nnUNetFrame")
nnDataset = os.path.join(nnFrame, "dataset")

# Configuración de variables de entorno
os.environ['nnUNet_raw'] = os.path.join(nnDataset, "nnUNet_raw")
os.environ['nnUNet_preprocessed'] = os.path.join(nnDataset, "nnUNet_preprocessed")
os.environ['nnUNet_results'] = os.path.join(nnDataset, "nnUNet_trained_models")
os.environ["base_path"] = os.path.join(wd, "segmentation")
os.environ["work_nnUNet"] = wd


# Rutas específicas del dataset


dataStroke = os.path.join(os.environ['nnUNet_raw'], "Dataset001_Stroke")
dataLabels = os.path.join(dataStroke, "labelsTr")
dataImages = os.path.join(dataStroke, "imagesTr")

# Función para crear directorios si no existen
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Creación de directorios necesarios
directories = [
    os.environ['nnUNet_raw'],
    os.environ['nnUNet_preprocessed'],
    os.environ['nnUNet_results'],
    dataStroke,
    dataLabels,
    dataImages
]

for directory in directories:
    create_dir(directory)

Posteriormente será necesaria la instalación del paquete nnUNet. De forma estándar el paquete entrena 1000 épocas cada uno de los modelos, lo cual requiere una alta disponibilidad de recursos computacionales. Por este motivo es necesario limitar dichos requerimientos, a pesar de que esto pueda suponer un empeoramiento en las capacidades del modelo.

Para hacerlo, una vez que hemos descargado el paquete mediante el comando

```
git clone https://github.com/MIC-DKFZ/nnUNet.git
```

Debemos entrar en `nnunetv2/training/nnUNetTrainer/nnUNetTrainer.py` y modificar el parámetro `self.num_epochs = 1000` de la linea 152 para reducirlo al número solicitado. En mi caso lo he reducido a 200 épocas


In [2]:
%%bash

# Navega al directorio donde se desea clonar el repositorio nnUNet.
cd /content/drive/MyDrive/nnUNetFrame

# Clona el repositorio nnUNet desde GitHub.
# Descomentar la siguiente línea si es la primera vez que se clona el repositorio.
# git clone https://github.com/MIC-DKFZ/nnUNet.git

# Cambia al directorio del repositorio nnUNet.
cd nnUNet

# Instala nnUNet en modo editable.
pip install -e .

# Vuelve al directorio nnUNetFrame.
cd ..

# Actualiza el paquete hiddenlayer desde el repositorio de GitHub de FabianIsensee.
pip install --upgrade git+https://github.com/FabianIsensee/hiddenlayer.git

# Instala el paquete ants.
pip install ants

# Instala el paquete antspyx.
pip install antspyx

# Instala el paquete intensity_normalization.
pip install intensity_normalization


Obtaining file:///content/drive/MyDrive/nnUNetFrame/nnUNet
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7

  Running command git clone --filter=blob:none --quiet https://github.com/FabianIsensee/hiddenlayer.git /tmp/pip-req-build-efdj3j_f


In [4]:
import torch
import SimpleITK as sitk
import ants
import shutil
import matplotlib.pyplot as plt
import nibabel as nib
import glob
import re
import numpy as np
from tqdm import tqdm
import multiprocessing
from intensity_normalization.typing import Modality, TissueType
from intensity_normalization.normalize.nyul import NyulNormalize

# Imprimir la versión de PyTorch
print("Pytorch version：")
print(torch.__version__)

# Imprimir la versión de CUDA
print("CUDA Version: ")
print(torch.version.cuda)

# Imprimir la versión de cuDNN
print("cuDNN version is :")
print(torch.backends.cudnn.version())


def prepare_dataset(origin_path: str, template_type: str = "FLAIR_template.nii.gz") -> None:
    """
    Prepares the dataset by creating necessary directories and calling preprocessing functions.

    Parameters:
    - origin_path (str): The path to the original dataset.
    - template_type (str): The name of the template file.

    Returns:
    - None
    """

    template = os.path.join(os.environ["base_path"], template_type)
    dest_path = os.path.join(os.environ["nnUNet_raw"], "Dataset001_Stroke")
    path_images = os.path.join(dest_path, "imagesTr")
    path_labels = os.path.join(dest_path, "labelsTr")

    # Crear directorios si no existen
    if not os.path.exists(path_images):
        os.makedirs(path_images, exist_ok=True)
        os.mkdir(path_labels)

    diccionario = emparejar(origin_path, dest_path)
    preprocess(diccionario, template)


def buscar_por_patron(lista: list, caso: str) -> str:
    """
    Searches for a file matching a specific pattern in a list.

    Parameters:
    - lista (list): List of filenames to search in.
    - caso (str): Case identifier to search for.

    Returns:
    - str: The filename that matches the pattern, or None if not found.
    """

    # En este caso seguimos la estructura de los datos de ISLES
    string = f'sub-strokecase{caso}_ses-0001_msk'
    pattern = re.compile(string)

    for elemento in lista:
        if pattern.search(elemento):
            return elemento
    return None


def emparejar(base_path: str, dest_path: str) -> dict:
    """
    Pairs FLAIR images with their corresponding mask files.

    Parameters:
    - base_path (str): The path to the base directory containing the FLAIR and mask files.
    - dest_path (str): The destination path where the paired files will be stored.

    Returns:
    - dict: A dictionary containing the paired files.
    """
    flair_files = glob.glob(os.path.join(base_path, "FLAIR_reg", "*"))
    mask_files = glob.glob(os.path.join(base_path, "LABELS_reg", "*"))

    resultado = {}
    pattern = re.compile(r'sub-strokecase(\d+)_ses-\d+_(FLAIR|msk)\.nii\.gz')

    for image_orig in flair_files:
        encont = pattern.search(image_orig)
        caso = encont.group(1)
        label_orig = buscar_por_patron(mask_files, caso)

        resultado[caso] = {
            'image': {
                'origin': image_orig,
                'dest': os.path.join(dest_path, 'imagesTr', f'ISLES_{caso}_0000.nii.gz')
            },
            'label': {
                'origin': label_orig,
                'dest': os.path.join(dest_path, 'labelsTr', f'ISLES_{caso}.nii.gz')
            }
        }

    return resultado


def convert_labels_to_nnUNet(image: sitk.Image) -> sitk.Image:
    """
    Converts label images to the nnUNet format.

    Parameters:
    - image (sitk.Image): The input label image.

    Returns:
    - sitk.Image: The converted label image.
    """
    img_npy = sitk.GetArrayFromImage(image)
    seg_new = np.zeros_like(img_npy)
    seg_new[img_npy >= 1] = 1
    seg_new[img_npy < 1] = 0
    img_corr = sitk.GetImageFromArray(seg_new)
    img_corr.CopyInformation(image)
    return img_corr

def bias_field_correct(raw_img_sitk: sitk.Image) -> sitk.Image:
    """
    Performs bias field correction on an input image.

    Parameters:
    - raw_img_sitk (sitk.Image): The input image to be corrected.

    Returns:
    - sitk.Image: The bias field corrected image.
    """
    raw_img_sitk_arr = sitk.GetArrayFromImage(raw_img_sitk)
    transformed = sitk.RescaleIntensity(raw_img_sitk, 0, 255)
    transformed = sitk.LiThreshold(transformed, 0, 1)
    head_mask = transformed
    shrinkFactor = 4
    inputImage = raw_img_sitk

    inputImage = sitk.Shrink(raw_img_sitk, [shrinkFactor] * inputImage.GetDimension())
    maskImage = sitk.Shrink(head_mask, [shrinkFactor] * inputImage.GetDimension())
    bias_corrector = sitk.N4BiasFieldCorrectionImageFilter()
    corrected = bias_corrector.Execute(inputImage, maskImage)
    log_bias_field = bias_corrector.GetLogBiasFieldAsImage(raw_img_sitk)
    corrected_image_full_resolution = raw_img_sitk / sitk.Exp(log_bias_field)

    return corrected_image_full_resolution


def fit_normalizer(standard_histogram_path: str) -> NyulNormalize:
    """
    Fits a normalizer using the Nyul normalization method.

    Parameters:
    - standard_histogram_path (str): Path to save or load the standard histogram.

    Returns:
    - NyulNormalize: The fitted normalizer.
    """
    normalizer = NyulNormalize()
    if os.path.exists(standard_histogram_path):
        normalizer.load_standard_histogram(standard_histogram_path)
    else:
        images_all = glob.glob(os.path.join(os.environ["base_path"], "FLAIR", "*"))
        images = [nib.load(path).get_fdata() for path in images_all]
        normalizer.fit(images)
        del images
        normalizer.save_standard_histogram(standard_histogram_path)
    return normalizer

def IntensityNormalization(image: sitk.Image, model: NyulNormalize) -> sitk.Image:
    """
    Normalizes the intensity of an image using a given model.

    Parameters:
    - image (sitk.Image): The input image.
    - model (NyulNormalize): The normalization model.

    Returns:
    - sitk.Image: The intensity-normalized image.
    """
    image_arr = sitk.GetArrayFromImage(image)
    new_image = model(image_arr)
    new_image = sitk.GetImageFromArray(new_image)
    new_image.CopyInformation(image)
    return new_image



def process_case(args: tuple) -> None:
    """
    Processes a single case by applying bias correction, intensity normalization, and label conversion.

    Parameters:
    - args (tuple): A tuple containing case identifier, data dictionary, and template path.

    Returns:
    - None
    """
    caso, datos, template_path = args

    # Cargar las imágenes
    image_raw = sitk.ReadImage(datos[caso]["image"]["origin"], sitk.sitkFloat32)
    label_raw = sitk.ReadImage(datos[caso]["label"]["origin"], sitk.sitkFloat32)
    template_read = ants.image_read(template_path, reorient='IAL')

    # Paso 1: Corrección de campo de sesgo
    image_bias = bias_field_correct(image_raw)

    # Paso 2: Normalización de intensidades
    norm_histo = os.path.join(os.environ["base_path"], "standard_histogram.npy")
    normalizer = fit_normalizer(norm_histo)
    image_trans = IntensityNormalization(image_bias, normalizer)

    # Paso 3: Corrección de etiquetas
    label_corr = convert_labels_to_nnUNet(label_raw)

    # Guardar las imágenes procesadas
    sitk.WriteImage(label_corr, datos[caso]["label"]["dest"])
    sitk.WriteImage(image_trans, datos[caso]["image"]["dest"])



def preprocess(datos_dic: dict, template_path: str) -> None:
    """
    Preprocesses the dataset using multiprocessing.

    Parameters:
    - datos_dic (dict): Dictionary containing data information.
    - template_path (str): Path to the template file.

    Returns:
    - None
    """
    num_cores = multiprocessing.cpu_count()  # Número de núcleos disponibles

    # Crear un pool de procesos
    with multiprocessing.Pool(num_cores) as pool:
        args_list = [(caso, datos_dic, template_path) for caso in datos_dic.keys()]

        # Configurar la barra de progreso
        with tqdm(total=len(args_list), desc="Processing cases") as pbar:
            for result in pool.imap_unordered(process_case, args_list):
                pbar.update()  # Actualizar la barra de progreso por cada tarea completada


# Por comodidad, las imágenes fueron registradas previamente en el dataset

def register_image_mask(template_read: ants.ANTsImage, image_read: ants.ANTsImage, mask_read: ants.ANTsImage) -> tuple:
    """
    Registers the image and mask to the template.

    Parameters:
    - template_read (ants.ANTsImage): The template image.
    - image_read (ants.ANTsImage): The image to be registered.
    - mask_read (ants.ANTsImage): The mask to be registered.

    Returns:
    - tuple: Registered mask and image.
    """
    transformation = ants.registration(
        fixed=template_read,
        moving=image_read,
        type_of_transform='Rigid',
        verbose=False
    )

    registered_img = transformation['warpedmovout']

    registered_mask_img = ants.apply_transforms(
        moving=mask_read,
        interpolator="bSpline",
        fixed=transformation['warpedmovout'],
        transformlist=transformation['fwdtransforms'],
        verbose=False
    )

    return registered_mask_img, registered_img

def itk_2_ants(image: sitk.Image) -> ants.ANTsImage:
    """
    Converts an ITK image to an ANTs image.

    Parameters:
    - image (sitk.Image): The ITK image to convert.

    Returns:
    - ants.ANTsImage: The converted ANTs image.
    """
    image_ants = ants.from_numpy(sitk.GetArrayFromImage(image).T,
                                 origin=image.GetOrigin(),
                                 spacing=image.GetSpacing(),
                                 direction=np.array(image.GetDirection()).reshape(3, 3))
    return image_ants


Pytorch version：
2.3.0+cu121
CUDA Version: 
12.1
cuDNN version is :
8902


## **Preparación del dataset**

Para la preparación del dataset se realizaron los siguientes pasos:

1.- **Registro de imágenes (normalización espacial)**: El registro de imágenes consiste en transportar todas las imágenes a un espacio común que permita realizar el análisis. Por comodidad a la hora de la instalación de diferentes softwares, el proceso fue realizado en un ordenador local. Dentro del registro, se realizó un registro al espacio MNI (Montreal Neurological Institute) que es un estándar dentro de la neuroimagen. Para el registro se realizó un registro rígido con 6 grados de libertad de las imágenes FLAIR, y posteriormente se aplicaron las transformaciones a las correspondientes máscaras. El registro fue realizado mediente el comando FLIRT contenido en el paquete FSL y se realizó una interpolación nearestNeighbour para las máscaras. Para la evalución de la calidad de los registros se empleó la herramienta slicesdir.

A partir de este paso de registro, las imágenes fueron subidas a Google Drive y están accesibles en este link

2.- **Corrección del sesgo de campo (bias field correction)**: debido a las inhomogeneidades del campo magnético producido por la resonancia magnética es necesario realizar un paso de normalización que corrige estas inhomogeneidades que afectan a la intensidad de la señal. Esto se realizó mediante la corrección N4 del paquete SimpleITK

3.- **Normalización de intensidad**: para la normalización de la intensidad de señal se empleó el método de Nyul. El método de Nyul se basa en la idea de ajustar las intensidades de las imágenes de tal manera que las distribuciones de intensidad de diferentes imágenes se alineen con una distribución estándar predefinida. Esto se hace en varias etapas:

- Selección de una plantilla: Se selecciona una imagen o un conjunto de imágenes que sirven como plantilla o estándar para la normalización.

- Cálculo del histograma estándar: Se calcula un histograma de intensidades para la imagen plantilla. Este histograma sirve como la distribución de referencia a la que se ajustarán las demás imágenes. Se realizó dicha normalización en base a todas las imágenes del dataset

- Transformación de las intensidades: Se determinan las transformaciones necesarias para ajustar las intensidades de una imagen dada a la distribución de intensidades de la plantilla. Esto implica mapear los percentiles de intensidad de la imagen original a los percentiles correspondientes en el histograma estándar.

- Aplicación de la transformación: Las intensidades de la imagen original se transforman de acuerdo con la función de mapeo calculada, resultando en una imagen con una distribución de intensidades alineada con la plantilla.

Las ventajas de la normalización de la intensidad de Nyul es que permite reducir la variabilidad intersujeto y así mejorar los resultados de los datos.

4.- **Comprobación de la integridad de las etiquetas de las máscaras**: Finalmente nos aseguramos que cumple correctamente el formato adecuado para nnUNET.

El código anterior también asegura que se mantiene de forma íntegra la estructura de los datos necesarios para el procesamiento por nnUNet

```
nnUNet_raw/Dataset001_NAME1
├── dataset.json
├── imagesTr
│   ├── ...
├── imagesTs
│   ├── ...
└── labelsTr
    ├── ...
nnUNet_raw/Dataset002_NAME2
├── dataset.json
├── imagesTr
│   ├── ...
├── imagesTs
│   ├── ...
└── labelsTr
    ├── ...
```



In [ ]:
# Preparamos el dataset requerido

prepare_dataset(os.environ["base_path"])

Processing cases: 100%|██████████| 250/250 [05:57<00:00,  1.43s/it]


## Preparación de los planes de nnUNet

Para ejecutar el entrenamiento es necesario que se genere un archivo .json que permita interpretar qué planes debe de ejecutar el algoritmo, así como algunas estadísticas.

In [ ]:
# Cambiar el directorio de trabajo al directorio nnUNet dentro del directorio MyDrive
os.chdir(os.path.join(nnFrame, "nnUNet"))

# Importaciones
from nnunetv2.dataset_conversion.generate_dataset_json import generate_dataset_json
from batchgenerators.utilities.file_and_folder_operations import *
from typing import Tuple

# Definir el directorio base de salida usando una variable de entorno
out_base = join(os.environ['nnUNet_raw'], foldername)

# Definir las rutas para las carpetas de imágenes de entrenamiento y etiquetas
imagestr = join(out_base, "imagesTr")
labelstr = join(out_base, "labelsTr")

# Obtener una lista de todos los casos de entrenamiento (archivos) en la carpeta de imágenes de entrenamiento
case_ids = glob.glob(join(imagestr, "*"))

# Generar el archivo JSON del dataset con la función generate_dataset_json
generate_dataset_json(
    out_base,                           # Ruta base de salida
    channel_names={"0": 'noNorm'},      # Nombre del canal (en este caso, sin normalización)
    labels={                            # Etiquetas del dataset con sus respectivos valores
        'background': 0,
        'infarct': 1
    },
    tensorImageSize="3D",               # Indicar que las imágenes son 3D
    num_training_cases=len(case_ids),   # Número de casos de entrenamiento
    file_ending='.nii.gz',              # Extensión de los archivos de imagen
    dataset_release='1.0'               # Versión del dataset
)

## Comprobando la integridad del dataset

Tras esto, el paquete nnUNet permite comprobar la integridad del dataset propuesto

In [9]:
%%bash

# Cambia el directorio de trabajo al directorio nnUNet_raw dentro del directorio dataset
cd $nnUNet_raw

# Ejecuta el comando nnUNetv2_plan_and_preprocess con varios parámetros:
# -d 1: Especifica el número del dataset (1 en este caso).
# -npfp 12: Define el número de procesos paralelos para la planificación (12 en este caso).
# --verify_dataset_integrity: Verifica la integridad del dataset antes de la planificación y el preprocesamiento.
# -pl nnUNetPlannerResEncL: Especifica el planificador a utilizar (nnUNetPlannerResEncL en este caso).

nnUNetv2_plan_and_preprocess -d 1 -npfp 12 --verify_dataset_integrity -pl nnUNetPlannerResEncL

Process is terminated.


## Entrenamiento del modelo

Esto permitirá entrenar el modelo solicitado. Podemos entrenarlo 5 veces, seleccionando el tipo de configuración solicitada. Mediante el flag --np se guardan las predicciones para posteriormente poder hacer un ensambling

In [ ]:
%%bash

## Entrenamos durante 5 folds el algoritmo

# Entrena el modelo nnUNet para el dataset 1 (Stroke) utilizando
# la configuración 3d_fullres y el planificador nnUNetResEncUNetLPlans.
# El entrenamiento se ejecuta en cinco diferentes pliegues (folds)
# para realizar validación cruzada.
# La opción --npz se utiliza para guardar las predicciones como archivos .npz.


nnUNetv2_train 1 3d_fullres 0 -p nnUNetResEncUNetLPlans --npz &&
nnUNetv2_train 1 3d_fullres 1 -p nnUNetResEncUNetLPlans --npz &&
nnUNetv2_train 1 3d_fullres 2 -p nnUNetResEncUNetLPlans --npz &&
nnUNetv2_train 1 3d_fullres 3 -p nnUNetResEncUNetLPlans --npz &&
nnUNetv2_train 1 3d_fullres 4 -p nnUNetResEncUNetLPlans --npz

## Búsqueda de la mejor configuración

En el siguiente comando permite determinar cuál es la mejor configuración del modelo

In [ ]:
# Ejecuta el comando nnUNetv2_find_best_configuration para encontrar
# la mejor configuración de entrenamiento para el dataset 1.
# La opción -c 3d_fullres especifica que se debe considerar la configuración
# en 3D.
# La opción -p nnUNetResEncUNetLPlans indica que se debe utilizar
# el planificador nnUNetResEncUNetLPlans.

!nnUNetv2_find_best_configuration 1 -c 3d_fullres -p nnUNetResEncUNetLPlans


***All results:***
nnUNetTrainer__nnUNetResEncUNetLPlans__2d: 0.4146338413159022

*Best*: nnUNetTrainer__nnUNetResEncUNetLPlans__2d: 0.4146338413159022

***Determining postprocessing for best model/ensemble***
Removing all but the largest foreground region did not improve results!

***Run inference like this:***

nnUNetv2_predict -d Dataset001_Stroke -i INPUT_FOLDER -o OUTPUT_FOLDER -f  0 1 2 3 4 -tr nnUNetTrainer -c 2d -p nnUNetResEncUNetLPlans

***Once inference is completed, run postprocessing like this:***

nnUNetv2_apply_postprocessing -i OUTPUT_FOLDER -o OUTPUT_FOLDER_PP -pp_pkl_file /content/drive/MyDrive/nnUNetFrame/dataset/nnUNet_trained_models/Dataset001_Stroke/nnUNetTrainer__nnUNetResEncUNetLPlans__2d/crossval_results_folds_0_1_2_3_4/postprocessing.pkl -np 8 -plans_json /content/drive/MyDrive/nnUNetFrame/dataset/nnUNet_trained_models/Dataset001_Stroke/nnUNetTrainer__nnUNetResEncUNetLPlans__2d/crossval_results_folds_0_1_2_3_4/plans.json


## Predicción de los resultados y postprocesado

Para finalmente validar los resultados encontrados en el dataset en cuestión realizaremos prueba con los datos proporcionados. Estos se almacenarán en una carpeta que se llamará test, e input

In [ ]:
%%bash

# Crear las carpetas de entrada y salida si no existen usando variables de entorno
mkdir -p $work_nnUNet/test/input
mkdir -p $work_nnUNet/test/output

Debemos depositar las imágenes que queremos dentro de la carpeta input

In [ ]:
%%bash

# Ejecuta el comando nnUNetv2_predict para realizar predicciones en el dataset especificado.
# La opción -d especifica el dataset.
# La opción -i especifica el directorio de entrada.
# La opción -o especifica el directorio de salida.
# La opción -f especifica los pliegues (folds) a utilizar.
# La opción -tr especifica el entrenador.
# La opción -c especifica la configuración (3D en este caso).
# La opción -p especifica el planificador a utilizar.

!nnUNetv2_predict \
    -d Dataset001_Stroke \
    -i $work_nnUNet/test/input \
    -o $work_nnUNet/test/output \
    -f 0 1 2 3 4 \
    -tr nnUNetTrainer \
    -c 3d_fullres \
    -p nnUNetResEncUNetLPlans

Finalmente aplicamos el postprocesamiento

In [ ]:
%%bash

# Ejecuta el comando nnUNetv2_apply_postprocessing para aplicar
# postprocesamiento a las predicciones generadas.
# La opción -i especifica el directorio de entrada donde se
# encuentran las predicciones.
# La opción -o especifica el directorio de salida donde se
# guardarán los resultados postprocesados.
# La opción -pp_pkl_file especifica el archivo .pkl que contiene
# la configuración de postprocesamiento.
# La opción -np especifica el número de procesos paralelos a
# utilizar.
# La opción -plans_json especifica el archivo JSON con los planes
# utilizados durante el entrenamiento.

nnUNetv2_apply_postprocessing \
    -i $work_nnUNet/test/output \
    -o $work_nnUNet/test/output_pp \
    -pp_pkl_file $nnUNet_results/Dataset001_Stroke/ \
    nnUNetTrainer__nnUNetResEncUNetLPlans__3d_fullres/ \
    crossval_results_folds_0_1_2_3_4/postprocessing.pkl \
    -np 8 \
    -plans_json $nnUNet_results/ \
    Dataset001_Stroke/nnUNetTrainer__nnUNetResEncUNetLPlans__3d_fullres/ \
    crossval_results_folds_0_1_2_3_4/plans.json


In [ ]:
import shutil

outputfile = "/content/drive/MyDrive/data_norm"
shutil.make_archive(outputfile, 'zip', "/content/nnUNetFrame/dataset/nnUNet_raw/Dataset001_Stroke")

'/content/drive/MyDrive/data_norm.zip'